# Homework 6, Part Two: A dataset about dogs.

Data from [a FOIL request to New York City](https://www.muckrock.com/foi/new-york-city-17/pet-licensing-data-for-new-york-city-23826/)

## Do your importing and your setup

In [ ]:
import pandas as pd
import openpyxl

## Read in the file `NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx` and look at the first five rows

In [ ]:
df = pd.read_excel("NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx",
                  nrows=30000,
                  na_values=["Unknown","UNKNOWN"])
df.columns = df.columns.str.replace(' ','_')

In [ ]:
df.head(5)

## How many rows do you have in the data? What are the column types?

If there are more than 30,000 rows in your dataset, go back and only read in the first 30,000.

In [ ]:
df.shape

In [ ]:
df.dtypes

## Describe the dataset in words. What is each row? List two column titles along with what each of those columns means.

For example: “Each row is an animal in the zoo. `is_reptile` is whether the animal is a reptile or not”

In [ ]:
#Each row is a lisenced dog in New York City as of 4-28-2016
#Primary Breed: the main breed of that dog
#Spayed or Neut: whether the dog is spayed/neut or not
#License Issued Date: when the license is issued

# Your thoughts

Think of four questions you could ask this dataset. **Don't ask them**, just write them down in the cell below. Feel free to use either Markdown or Python comments.

In [ ]:
#1. What's the average age when the dog get an issued liscence?
#2. How long it will take to get an application of a dog liscence approved?
#3. Which area (basically zip code) has the most licensed dogs?
#4. Is there more female dogs or male dogs?

# Looking at some dogs

## What are the most popular (primary) breeds of dogs? Graph the top 10.

In [ ]:
df.Primary_Breed.value_counts().head(10).plot(kind='bar')

## "Unknown" is a terrible breed! Graph the top 10 breeds that are NOT Unknown

In [ ]:
df.Primary_Breed.value_counts().head(10).plot(kind='bar')

## What are the most popular dog names?

In [ ]:
df.Animal_Name.value_counts().head(5)

## Do any dogs have your name? How many dogs are named "Max," and how many are named "Maxwell"?

In [ ]:
df[df['Animal_Name']=='']

In [ ]:
len(df[df['Animal_Name']=='Max'])

In [ ]:
len(df[df['Animal_Name']=='Maxwell'])

## What percentage of dogs are guard dogs?

Check out the documentation for [value counts](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html).

In [ ]:
df.Guard_or_Trained.value_counts(normalize=True)

## What are the actual numbers?

In [ ]:
df.Guard_or_Trained.value_counts()

## Wait... if you add that up, is it the same as your number of rows? Where are the other dogs???? How can we find them??????

Use your `.head()` to think about it, then you'll do some magic with `.value_counts()`

In [ ]:
df.Guard_or_Trained.value_counts(dropna=False)

## Fill in all of those empty "Guard or Trained" columns with "No"

Then check your result with another `.value_counts()`

In [ ]:
df.Guard_or_Trained.fillna("No", inplace = True)
df.Guard_or_Trained.value_counts()

## What are the top dog breeds for guard dogs? 

In [ ]:
guard_dog = df[df['Guard_or_Trained'] == 'Yes']
guard_dog.Primary_Breed.value_counts()

## Create a new column called "year" that is the dog's year of birth

The `Animal Birth` column is a datetime, so you can get the year out of it with the code `df['Animal Birth'].apply(lambda birth: birth.year)`.

In [ ]:
df['year'] = df['Animal_Birth'].apply(lambda birth: birth.year)
df.head(5)

## Calculate a new column called “age” that shows approximately how old the dog is. How old are dogs on average?

In [ ]:
#since the data set is from 2016, I think it's more meaningful to calculate the age of the dog when the license is issued rather than its current age
df['age'] = df['License_Issued_Date'].apply(lambda birth: birth.year) - df['year']
df.age.mean()

# Joining data together

## Which neighborhood does each dog live in?

You also have a (terrible) list of NYC neighborhoods in `zipcodes-neighborhoods.csv`. Join these two datasets together, so we know what neighborhood each dog lives in. **Be sure to not read it in as `df`, or else you'll overwrite your dogs dataframe.**

In [ ]:
data2 = pd.read_csv('zipcodes-neighborhoods.csv')
data2.rename(columns={'zip':'Owner_Zip_Code'}, inplace = True)
df2 = pd.merge(df, data2,
              on='Owner_Zip_Code',
              how='left')
df2.head(5)

## What is the most popular dog name in all parts of the Bronx? How about Brooklyn? The Upper East Side?

In [ ]:
bronx_dog = df2[df2['borough'] == 'Bronx']
bronx_dog.Animal_Name.value_counts()

In [ ]:
brooklyn_dog = df2[df2['borough'] == 'Brooklyn']
brooklyn_dog.Animal_Name.value_counts()

In [ ]:
ues_dog = df2[df2['neighborhood'] == 'Upper East Side']
ues_dog.Animal_Name.value_counts()

## What is the most common dog breed in each of the neighborhoods of NYC?

In [ ]:
df2.groupby(by='neighborhood').Primary_Breed.value_counts()

## What breed of dogs are the least likely to be spayed? Male or female?

In [ ]:
#I want to sort only with 'No' but I can't make it
df2.groupby(by='Primary_Breed').Spayed_or_Neut.value_counts(normalize=True).sort_values()

In [ ]:
#to know the gender, I can filter out the breed with no spayed/neut, and then do .gender.value_counts()

## Make a new column called monochrome that is True for any animal that only has black, white or grey as one of its colors. How many animals are monochrome?

## How many dogs are in each borough? Plot it in a graph.

In [ ]:
df2.groupby(by='borough').Owner_Zip_Code.count().plot(kind='bar')

## Which borough has the highest number of dogs per-capita?

You’ll need to merge in `population_boro.csv`

In [ ]:
data3 =  pd.read_csv('boro_population.csv')
df3 = pd.merge(df2, data3,
              on='borough',
              how='left')
dogs_number = df3.groupby(by='borough').Owner_Zip_Code.count()
population = df3.groupby(by='borough').population.mean()
(dogs_number/population).sort_values(ascending=False)

## Make a bar graph of the top 5 breeds in each borough.

How do you groupby and then only take the top X number? You **really** should ask me, because it's kind of crazy.

## What percentage of dogs are not guard dogs?